<a href="https://colab.research.google.com/github/ClarenceBrn/Fairness/blob/main/MiProjet_Bernabe_Clarence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MiProjet Bernabé Clarence



## Introduction
Pour ce projet, nous allons analyser un sous-ensemble des métasdonnées de 15000 individus, issus du jeu de données Chest X ray NIH 14 Dataset : https://www.kaggle.com/datasets/nih-chest-xrays/data. Le fichier contenant les données de départ est Bernabe_Clarence.csv.

### Objectifs:
1. Analyse des métadonnées
2. Identification de biais (s'il en existe)
3. Réduction des biais grâce à une méthode de pré-processing (nouveau dataset avec biais réduits)

### Présentation du dataset
Ce dataset contient 53467 lignes, chacune avec 11 features qui sont : Image Index, Finding Labels, Follow-up #, Patient ID, Patient Age, Patient Gender, View Position, OriginalImage[Width, Heigth], OriginalImagePixelSpacing[x, y].  
Chaque ligne correspond à un scanner différent, et ces scanners ont été effectués sur 15000 patients (Donc plusieurs scanners ont été effectués sur certains patients).  
La colonne Findings Labels correspond à la maladie dont souffre le patient.  
Follow-up # indique le rang chronologique de la radiographie pour un patient (0 = première visite, 1 = deuxième visite , ect...).  
View Position correspond à l'angle de pris de vue, PA (Postero-Anterior, debout) ou AP (Antero-Posterior, allongé).  


In [1]:
# To execute only in Colab
! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 69.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.8/191.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Code to compute fairness metrics using aif360

from aif360.sklearn.metrics import *
from sklearn.metrics import  balanced_accuracy_score


# This method takes lists
def get_metrics(
    y_true, # list or np.array of truth values
    y_pred=None,  # list or np.array of predictions
    prot_attr=None, # list or np.array of protected/sensitive attribute values
    priv_group=1, # value taken by the privileged group
    pos_label=1, # value taken by the positive truth/prediction
    sample_weight=None # list or np.array of weights value,
):
    group_metrics = {}
    group_metrics["base_rate_truth"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["base_rate_preds"] = base_rate(
        y_true=y_pred, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        if len(set(y_pred))>1:
            group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
                y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
            )
        else:
            group_metrics["conditional_demographic_disparity"] =None
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["balanced_accuracy_score"] = balanced_accuracy_score(
        y_true=y_true, y_pred=y_pred, sample_weight=sample_weight
        )
    return group_metrics

/usr/local/lib/python3.12/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.12/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [3]:
import numpy as np
import fairlearn
import plotly.express as px
import pandas as pd
#print(np.__version__,fairlearn.__version__)
print("Importation des bibliothèques réussi.\n")

Importation des bibliothèques réussi.



In [21]:
url = "https://raw.githubusercontent.com/ClarenceBrn/Fairness/main/Bernabe_Clarence.csv"
df = pd.read_csv(url)
print("Chargement du dataset sous forme dataframe réussi.\n")

Chargement du dataset sous forme dataframe réussi.



In [22]:
df


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143000,0.143000
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143000,0.143000
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168000,0.168000
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171000,0.171000
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143000,0.143000
...,...,...,...,...,...,...,...,...,...,...,...
53462,00030787_000.png,No Finding,0,30787,34,M,PA,2021,2021,0.194311,0.194311
53463,00030789_000.png,Infiltration,0,30789,52,F,PA,2021,2021,0.194311,0.194311
53464,00030797_000.png,No Finding,0,30797,24,M,PA,2021,2021,0.194311,0.194311
53465,00030798_000.png,No Finding,0,30798,30,M,PA,2500,2048,0.171000,0.171000


#### Modification du noms features pour faciliter la manipulation des données
Les nouveaux noms sont : ['Image_Index', 'Finding_Labels', 'Follow-up_#', 'Patient_ID',
       'Patient_Age', 'Patient_Gender', 'View_Position', 'OriginalImageWidth',
       'OriginalImageHeight', 'OriginalImagePixSpacingX', 'OriginalImagePixSpacingY'].

In [23]:
df = df.rename(columns={
    'OriginalImage[Width' : 'OriginalImageWidth',
    'Height]' : 'OriginalImageHeight',
    'OriginalImagePixelSpacing[x' : 'OriginalImagePixSpacingX',
    'y]' : 'OriginalImagePixSpacingY'
})
df.columns = df.columns.str.replace(' ', '_')
df.columns


Index(['Image_Index', 'Finding_Labels', 'Follow-up_#', 'Patient_ID',
       'Patient_Age', 'Patient_Gender', 'View_Position', 'OriginalImageWidth',
       'OriginalImageHeight', 'OriginalImagePixSpacingX',
       'OriginalImagePixSpacingY'],
      dtype='object')

## Préparation de la donnée

In [24]:
# On vérifie qu'il n'y ai pas de données manquantes
for column in df.columns:
  print("Données manquantes pour la feature ",column, " = ", df[column].isnull().values.any())

Données manquantes pour la feature  Image_Index  =  False
Données manquantes pour la feature  Finding_Labels  =  False
Données manquantes pour la feature  Follow-up_#  =  False
Données manquantes pour la feature  Patient_ID  =  False
Données manquantes pour la feature  Patient_Age  =  False
Données manquantes pour la feature  Patient_Gender  =  False
Données manquantes pour la feature  View_Position  =  False
Données manquantes pour la feature  OriginalImageWidth  =  False
Données manquantes pour la feature  OriginalImageHeight  =  False
Données manquantes pour la feature  OriginalImagePixSpacingX  =  False
Données manquantes pour la feature  OriginalImagePixSpacingY  =  False


In [25]:
print("Nombre de patient différent : " + str(df["Patient_ID"].nunique())+"\n")
print("Dimension du dataset : " + str(df.shape) + "\n")
print("Nombre de labels différent : " + str(df["Finding_Labels"].nunique()) + "\n")
print("Tranche d'âge des patients : " + str(df["Patient_Age"].min()) + " -> " + str(df["Patient_Age"].max()) + "\n")
print("Les différents genres de patients : " + str(df["Patient_Gender"].unique()) + "\n")
print("Les différents angles de prise de vue : " + str(df["View_Position"].unique()) + "\n")

Nombre de patient différent : 15000

Dimension du dataset : (53467, 11)

Nombre de labels différent : 603

Tranche d'âge des patients : 1 -> 414

Les différents genres de patients : ['M' 'F']

Les différents angles de prise de vue : ['PA' 'AP']



On remarque que les patients ont entre 1 et 414 ans. Cette borne supérieur est absurde, essayons de la corriger.

In [26]:
df[df.Patient_Age > 120]

,Image_Index,Finding_Labels,Follow-up_#,Patient_ID,Patient_Age,Patient_Gender,View_Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixSpacingX,OriginalImagePixSpacingY
9834,00005567_000.png,Effusion|Pneumonia,0,5567,412,M,AP,3056,2544,0.139,0.139
22095,00011973_002.png,Edema,2,11973,414,M,AP,3056,2544,0.139,0.139
22866,00012238_010.png,No Finding,10,12238,148,M,PA,2992,2991,0.143,0.143
30265,00015558_000.png,No Finding,0,15558,149,M,PA,2992,2991,0.143,0.143
35703,00018366_044.png,Pneumothorax,44,18366,152,F,PA,2302,2991,0.143,0.143
37603,00019346_000.png,Infiltration,0,19346,151,F,PA,2678,2774,0.143,0.143
40820,00021047_002.png,Mass|Pleural_Thickening,2,21047,412,M,AP,3056,2544,0.139,0.139
41197,00021275_003.png,No Finding,3,21275,413,F,AP,3056,2544,0.139,0.139
43627,00022811_000.png,No Finding,0,22811,412,M,PA,3056,2544,0.139,0.139
45685,00025206_000.png,Infiltration|Mass,0,25206,153,M,PA,2992,2991,0.143,0.143


Cherchons maintenant si les patients avec ces âges ont d'autres scanners avec cette fois un âge plus cohérent grâce à leurs Patient_Id.

In [27]:
ids_absurdes = df[df.Patient_Age > 120].Patient_ID.unique()
lignes_ids_absurdes = df[df.Patient_ID.isin(ids_absurdes)]
lignes_triees = lignes_ids_absurdes.sort_values(by=['Patient_ID', 'Follow-up_#'])
historique_ages = lignes_triees.groupby('Patient_ID')['Patient_Age'].apply(list)
#for id_patient, liste_ages in historique_ages.items():
    #print(f"Patient {id_patient} : {liste_ages}")

On va décider de supprimer les lignes avec un âge absurde lorsque les patients ne sont venus qu'une fois (seulement 3 lignes concernées), et sinon de modifier la valeur lorsqu'elle est absurde grâce aux valeurs des autres visites.

In [28]:
corrections = {
    5567: 53,
    11973: 61,
    12238: 64,
    15558: 46,
    18366: 64,
    21047: 52,
    21275: 21,
    22811: 25,
    25206: 36,
    26028: 60
}

for patient_id, new_age in corrections.items():
    df.loc[(df['Patient_ID'] == patient_id) & (df['Patient_Age'] > 100), 'Patient_Age'] = new_age

df = df[df['Patient_Age'] <= 100]
print("Tranche d'âge des patients : " + str(df["Patient_Age"].min()) + " -> " + str(df["Patient_Age"].max()) + "\n")

Tranche d'âge des patients : 1 -> 94



On a réussi à gérer les valeurs absurdes présentes dans le jeu de données.

On va maintenant modifier les colonnes catégorielles en colonnes numériques. Patient_Gender et View_Position ne possèdent que deux valeurs différentes chacunes, respectivement F et M, et PA et AP. Les colonnes Patient_Age et Follow-up_# vont également être binarisées pour avoir un groupe (potentiellement) privilégié et un non privilégié. On va les remplacer par 0 et 1 dans chacun des cas.

In [29]:
df['Patient_Gender'] = df['Patient_Gender'].apply(lambda x: 1 if x == 'M' else 0)
df['View_Position'] = df['View_Position'].apply(lambda x: 1 if x == 'PA' else 0)
df['Age_Binary'] = df['Patient_Age'].apply(lambda x: 1 if x < 60 else 0)
df['Followup_Binary'] = df['Follow-up_#'].apply(lambda x: 1 if x == 0 else 0)

In [30]:
df

,Image_Index,Finding_Labels,Follow-up_#,Patient_ID,Patient_Age,Patient_Gender,View_Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixSpacingX,OriginalImagePixSpacingY,Age_Binary,Followup_Binary
0,00000001_000.png,Cardiomegaly,0,1,58,1,1,2682,2749,0.143000,0.143000,1,1
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,1,1,2894,2729,0.143000,0.143000,1,0
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,1,1,2500,2048,0.168000,0.168000,1,0
3,00000002_000.png,No Finding,0,2,81,1,1,2500,2048,0.171000,0.171000,0,1
4,00000003_000.png,Hernia,0,3,81,0,1,2582,2991,0.143000,0.143000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53462,00030787_000.png,No Finding,0,30787,34,1,1,2021,2021,0.194311,0.194311,1,1
53463,00030789_000.png,Infiltration,0,30789,52,0,1,2021,2021,0.194311,0.194311,1,1
53464,00030797_000.png,No Finding,0,30797,24,1,1,2021,2021,0.194311,0.194311,1,1
53465,00030798_000.png,No Finding,0,30798,30,1,1,2500,2048,0.171000,0.171000,1,1


Enfin, comme vu au cours des différents TP, on va décider de binariser notre label, les 603 valeurs possibles deviendront alors 0 si l'on a No Finding, c'est à dire pas de maladie détéctée, et 1 sinon.

In [31]:
df['Label'] = df['Finding_Labels'].apply(lambda x: 0 if x == 'No Finding' else 1)
df['Label'].value_counts()

,count
Label,
0,28921
1,24543


In [33]:
print(f"Nombre de patients uniques malades : {df[df['Label'] == 1]['Patient_ID'].nunique()}")

Nombre de patients uniques malades : 6987


On peut notamment remarquer que le nombre de scanner de patients malades est relativement similaire à celui des non-malades, on n'a pas de fort déséquilibre d'une des classes. On peut faire la même observation pour le nombre de patients distincts malade, environ 7000 sur 15000, donc légerment moins que la moitié.

##  Analyse descriptive et observation des biais

On va maintenant essayer d'observer ou non des corrélations entre le fait d'être malade ou non (colonne Label), et les autres features.

In [34]:
def Compute_correlation(cola, colb):
  return np.corrcoef(df[cola].values, df[colb].values)[0][1]

In [35]:
features = df.columns.drop(['Image_Index', 'Finding_Labels', 'Patient_ID','Label'])

for feature in features:
  print(feature, Compute_correlation('Label', feature))

Follow-up_# 0.17087841118846517
Patient_Age 0.07779412268830725
Patient_Gender 0.009401117290829347
View_Position -0.1088064100866525
OriginalImageWidth 0.06341875238427651
OriginalImageHeight -0.006098067828047173
OriginalImagePixSpacingX -0.036907870687519544
OriginalImagePixSpacingY -0.036907870687519544
Age_Binary -0.06584496830098303
Followup_Binary -0.18443344294892888


Aucune corrélation forte semble pouvoir être tirée directement, la plus grande corrélation est la colonne Followup_Binary (qui indique s'il s'agit d'une première visite ou non), avec une corrélation de -0.184, suivi de Follow-up_# (qui indique la visite numéro x d'un patient), avec une corrélation de 0.171, ainsi que de View_position (l'angle de prise de vue) avec une corrélation de 0.109.

On va maintenant quantifier les disparités statistiques présentes dans la vérité terrain avant la modélisation, afin de détecter si le dataset contient des biais structurels.  
Les groupes "privilégiés" sont représentés par la valeur 1 pour les features qu'on souhaite tester (Genre, Age, Angle de pris de vue et Première visite).

In [38]:
m_gender = get_metrics(df['Label'], df['Label'], df['Patient_Gender'])
print(f"Genre (H=1 vs F=0) - Statistical Parity: {m_gender['statistical_parity_difference']:.4f}")

m_pos = get_metrics(df['Label'], df['Label'], df['View_Position'])
print(f"Position (PA=1 vs AP=0) - Statistical Parity: {m_pos['statistical_parity_difference']:.4f}")

m_age = get_metrics(df['Label'], df['Label'], df['Age_Binary'])
print(f"Age (<60=1 vs >=60=0) - Statistical Parity: {m_age['statistical_parity_difference']:.4f}")

m_followup = get_metrics(df['Label'], df['Label'], df['Followup_Binary'])
print(f"Follow_up (Première visite=1 vs Suivis=0) - Statistical Parity: {m_followup['statistical_parity_difference']:.4f}")

Genre (H=1 vs F=0) - Statistical Parity: -0.0095
Position (PA=1 vs AP=0) - Statistical Parity: 0.1113
Age (<60=1 vs >=60=0) - Statistical Parity: 0.0764
Follow_up (Première visite=1 vs Suivis=0) - Statistical Parity: 0.2046


On va maintenant entraîner un classifieur et calculer diférentes métriques de fairness, en fonction de différents attributs sensibles.

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[features], df['Label'], test_size=0.33, random_state=42)

In [37]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=42)

clf = clf.fit(X_train, y_train)
preds = clf.predict(X_test)
clf.score(X_test, y_test)

0.5669349353888007